<a href="https://colab.research.google.com/github/michael-wettach/pythonsamples/blob/main/xml_samples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Wir wollen eine XML Datei schreiben, die so aussieht:</br>
&lt;root&gt;</br>
 &lt;doc&gt;</br>
     &lt;field1 name="mein_feld1"&gt;irgendein Inhalt&lt;/field1&gt;</br>
     &lt;field2 name="mein_feld2"&gt;ein anderer Inhalt&lt;/field2&gt;</br>
 &lt;/doc&gt;</br>
&lt;/root&gt;</br>

Es gibt eine Reihe von XML Bibliotheken in Python. Die am meisten verwendete ist ElementTree, die hat auch eine performante C-Implementierung cELementTree. In Python ab Version 3.3 wird die automatisch gezogen, wenn man ElementTree benutzt, in früheren Python Versionen muss man sie explizit ansprechen. Die Doku dazu findet man hier: https://docs.python.org/3/library/xml.etree.elementtree.html

In [ ]:
import xml.etree.cElementTree as ET

# Definiere den obersten Knoten als XML ELement
root = ET.Element("root")

# Definiere den nächsten Knoten als Unterelement
doc = ET.SubElement(root, "doc")

# Und da hängen wir noch die zwei Felder drunter
ET.SubElement(doc, "field1", name="my_field1").text = "irgendein Inhalt"
ET.SubElement(doc, "field2", name="my_field2").text = "ein anderer Inhalt"

# Jetzt haben wir eine Struktur und machen einen XML Baum daraus
tree = ET.ElementTree(root)

# xml_declaration erzeugt einen Header
# ElementTree hat leider keine PrettyPrint Funktion.
tree.write("filename.xml", xml_declaration=True,encoding='utf-8', method="xml")

# Öffnen der gerade geschriebenen Datei
file = open('filename.xml')
for line in file.readlines():
  print(line)


<?xml version='1.0' encoding='utf-8'?>

<root><doc><field1 name="my_field1">irgendein Inhalt</field1><field2 name="my_field2">ein anderer Inhalt</field2></doc></root>


Ein Namespace bietet die Möglichkeit, XML Tags aus verschiedenen Applikationen zu mischen ohne dass diese verwechselt werden. Im obigen Beispiel ist noch kein Namespace deklariert. Das geht aber natürlich auch...

In [ ]:
import xml.etree.cElementTree as ET

# Hier definieren wir einen Namespace
ET.register_namespace('com','http://www.company.com') 
my_ns = r'{http://www.company.com}'

# Definiere den obersten Knoten als XML ELement
root = ET.Element(my_ns + "root")

# Definiere den nächsten Knoten als Unterelement
doc = ET.SubElement(root, my_ns + "doc")

# Und da hängen wir noch die zwei Felder drunter
ET.SubElement(doc, my_ns + "field1", name="my_field1").text = "irgendein Inhalt"
ET.SubElement(doc, my_ns + "field2", name="my_field2").text = "ein anderer Inhalt"

# Jetzt haben wir eine Struktur und machen einen XML Baum daraus
tree = ET.ElementTree(root)

# xml_declaration erzeugt einen Header
# ElementTree hat leider keine PrettyPrint Funktion.
tree.write("filename.xml", xml_declaration=True,encoding='utf-8', method="xml")

# Öffnen der gerade geschriebenen Datei
file = open('filename.xml')
for line in file.readlines():
  print(line)


<?xml version='1.0' encoding='utf-8'?>

<com:root xmlns:com="http://www.company.com"><com:doc><com:field1 name="my_field1">irgendein Inhalt</com:field1><com:field2 name="my_field2">ein anderer Inhalt</com:field2></com:doc></com:root>


Eine andere häufig verwendete Bibliothek ist LXML. Ein Tutorial findet man hier: https://lxml.de/tutorial.html  

In [ ]:
import lxml.etree
import lxml.builder    

# Zuerst erzeugen wir die XML Tags (Elementtypen)
E = lxml.builder.ElementMaker()
ROOT = E.root
DOC = E.doc
FIELD1 = E.field1
FIELD2 = E.field2

# Und dann die Dokumentstruktur
the_doc = ROOT(
        DOC(
            FIELD1('irgendein Inhalt', name='my_field1'),
            FIELD2('ein anderer Inhalt', name='my_field2'),
            )   
        )   

# Hier gibt es auch eine PrettyPrint Option
output = lxml.etree.tostring(the_doc, xml_declaration=True, encoding='utf8', method='xml', pretty_print=True)

# Zum Drucken verwende ich hier den * Operator, der eine Liste in ein Tupel auspackt 
# und die Elemente einzeln an die umgebende Funktion liefert. 
print(*output.splitlines(), sep="\n")

b"<?xml version='1.0' encoding='utf8'?>"
b'<root>'
b'  <doc>'
b'    <field1 name="my_field1">irgendein Inhalt</field1>'
b'    <field2 name="my_field2">ein anderer Inhalt</field2>'
b'  </doc>'
b'</root>'


In [ ]:
# In LXML kann man auch nachträglich Elemente ergänzen
# Zum obersten Knoten root
the_doc.append(FIELD2('noch ein Inhalt', name='my_field3'))

# Oder zu einem Knoten, den wir erst noch suchen
my_doc = the_doc.find("doc")
my_doc.append(FIELD2('noch ein Inhalt', name='my_field4'))

output = lxml.etree.tostring(the_doc, xml_declaration=True, encoding='utf8', method='xml', pretty_print=True)
print(*output.splitlines(), sep="\n")

b"<?xml version='1.0' encoding='utf8'?>"
b'<root>'
b'  <doc>'
b'    <field1 name="my_field1">irgendein Inhalt</field1>'
b'    <field2 name="my_field2">ein anderer Inhalt</field2>'
b'    <field2 name="my_field4">noch ein Inhalt</field2>'
b'  </doc>'
b'  <field2 name="my_field3">noch ein Inhalt</field2>'
b'</root>'
